In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import inflect
from tqdm import tqdm
import pandas as pd

# Download NLTK resources if not already downloaded1
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#download the csv file
! gdown -- 13pDeQeEWKopQSYVwr87YDbwV6PMZadPA

Downloading...
From (original): https://drive.google.com/uc?id=13pDeQeEWKopQSYVwr87YDbwV6PMZadPA
From (redirected): https://drive.google.com/uc?id=13pDeQeEWKopQSYVwr87YDbwV6PMZadPA&confirm=t&uuid=34c73a23-041a-475c-92f6-4ac56e1c503e
To: /content/eng_subtitles.csv
100% 5.01G/5.01G [00:44<00:00, 113MB/s]


### Observation
- This is a tip to download files from gdrive to google colab environment

# Mounting

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


# Read the pickle file

In [ ]:
df=pd.read_pickle('/content/chunk_1.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 60000 to 69999
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   num              10000 non-null  int64 
 1   name             10000 non-null  object
 2   decoded_content  10000 non-null  object
dtypes: int64(1), object(2)
memory usage: 234.5+ KB


In [ ]:
#df=df[['name','decoded_content']].reset_index().set_index('index')
df

,name,decoded_content
index,,
60000,black.sunday.(1977).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nUse the ..."
60001,hapritza.hagdola.(1970).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
60002,witness.in.the.war.zone.(1987).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\napi.Open..."
60003,disengagement.(2007).eng.1cd,"1\r\n00:00:44,734 --> 00:00:48,807\r\n- Can I ..."
60004,the.romantics.s01.e01.the.boy.from.jalandhar.(...,"ï»¿1\r\n00:00:06,791 --> 00:00:07,708\r\n[Katr..."
...,...,...
69995,sky.high.s01.e07.en.la.boca.del.lobo.(2023).en...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nAdver..."
69996,sky.high.s01.e01.los.vivos.y.los.muertos.(2023...,"ï»¿1\r\n00:00:11,166 --> 00:00:13,666\r\nINSPI..."
69997,sky.high.s01.e02.nuevas.amistades.viejos.enemi...,"ï»¿1\r\n00:07:15,000 --> 00:07:17,541\r\nSKY H..."


# Step-3 Preprocess the content

In [ ]:
##timestamp_pattern=r'\d+\r?\n\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}'
dialogue_pattern=r"Dialogue: (Marked=)?\d+,\d{1,2}:\d{2}:\d{2}\.\d{2},\d{1,2}:\d{2}:\d{2}\.\d{2},\*?(\w+)?,(\w+)?,\d{1,4},\d{1,4},\d{1,4},,"
header_pattern1=r'Watch any video online with Open-SUBTITLES\r\nFree Browser extension: osdb.link/ext\r\n\r\n\r\n'#header
header_pattern2=r'\r\nAdvertise your product or brand here\r\ncontact www.OpenSubtitles.org today'
footer_pattern=r'Please rate this subtitle at www.osdb.link/[0-9a-z]+\r\nHelp other users to choose the best subtitles' #footer
header_pattern = r'\[Script Info\].*?\[Events\]\r\n.*?\r\n(Subtitle:)?'
link_pattern=r'(https?:\/\/)?(www\.)?(\w+)(\.\w+)'
linebreak_pattern=r'(\r?\n)+'
punctuation_pattern=r'[^a-zA-Z]'

In [ ]:

    def text_preprocessing(corpus,flag):


        if re.search(timestamp_pattern,corpus): #movie or tv show
            #remove timestamps
            corpus=re.sub(timestamp_pattern,'',corpus)
            #remove header and footer
            corpus=re.sub(header_pattern1,'',corpus)
            corpus=re.sub(footer_pattern,'',corpus)

        else: #anime
            #removing header
            corpus=re.sub(header_pattern,'',corpus,flags=re.DOTALL)
            #removing dialogue
            corpus=re.sub(dialogue_pattern,'',corpus)

        #remove linebreaks
        corpus=re.sub(linebreak_pattern,' ',corpus)

        #remove links
        corpus=re.sub(link_pattern,'',corpus)

        #remove punctuation
        corpus=re.sub(punctuation_pattern,' ',corpus)



        #convert to lower case
        corpus=corpus.lower()

        if flag == "stemming":
            stemmer = SnowballStemmer(language='english')
            preprocess_func = stemmer.stem
        else:
            lemmatizer = WordNetLemmatizer()
            preprocess_func = lemmatizer.lemmatize

        # Tokenize text
        words = word_tokenize(corpus.lower())

        # Precompute stopwords
        stop_words = set(nltk.corpus.stopwords.words('english'))

        # Process tokens
        processed_words = [preprocess_func(word) for word in words if word not in stop_words]

        # Join processed words
        return ' '.join(processed_words)

In [ ]:
tqdm.pandas()
df['pre-processed_content'] = df['decoded_content'].progress_apply(lambda x: text_preprocessing(x, "stemming"))



100%|██████████| 10000/10000 [10:10<00:00, 16.39it/s]


In [ ]:
new_df=df[['name','pre-processed_content']]
new_df

,name,pre-processed_content
index,,
50000,king.of.the.hill.s04.e10.hillennium.(1999).eng...,boy tell chappi got good look tree year dig ca...
50001,king.of.the.hill.s04.e11.old.glory.(2000).eng.1cd,advertis product brand contact today oh shoot ...
50002,king.of.the.hill.s04.e12.rodeo.days.(2000).eng...,nice countri mayb retir tell em go hell may se...
50003,king.of.the.hill.s04.e13.hanky.panky.(2000).en...,advertis product brand contact today yeah righ...
50004,king.of.the.hill.s04.e14.high.anxiety.(2000).e...,previous king hill public littl bimbo need pro...
...,...,...
59995,rock.the.casbah.(2013).eng.1cd,fez littl go movi everi saturday brother ahm m...
59996,des.gens.qui.sembrassent.(2013).eng.1cd,happen saint tropez grandpa glad see physic fi...
59997,foreign.letters.(2012).eng.1cd,hi must elli want put bag coat room want hang ...


In [ ]:
try:
  new_df.to_csv('/content/drive/My Drive/Colab Notebooks/50000-60000eng_subtitles.csv')
  new_df.info()
except Exception as e:
  print(e)

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 50000 to 59999
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   name                   10000 non-null  object
 1   pre-processed_content  10000 non-null  object
dtypes: object(2)
memory usage: 234.4+ KB


### Observation
- Saving file directly to a gdrive, since download speeds within google colab are dead slow compared to gdrive